In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import wget
import os
import zipfile
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from numpy import random
from scipy.stats import binom
import scipy.stats as stats
from statsmodels.tsa.arima_process import ArmaProcess
from scipy.special import digamma,polygamma
from scipy.linalg import sqrtm
from tqdm import tqdm
from datetime import datetime

In [ ]:
from nonparam_compositional_cpd.nonparametric_beta_ar_change_point_tester import NonparametricBetaARChangePointTester
from nonparam_compositional_cpd.threshold_generator import ThresholdGenerator

# Method comparison

In [ ]:
# Nonparametric change-point detection

In [ ]:
m=400
N= np.round(len(Y)/m-1, 2)
grid = 5
length = 10
nIteration = 500
gamma_vector = [0,0.25,0.4]
A = np.eye(grid)

detected_point = []
for i in range(200, 300):
    if i%5==0:
        print(F"Processing the {str(i)} dataset.")
    Y = pd.read_csv(f"{os.getcwd()}/ds/Ha_X_datasets/X_seed_{str(i)}.csv")
    Y.columns = ["per"]
    yrend_in = 2010
    Y["year"] = [yrend_in]*m + [yrend_in+10] * (len(Y)-m)

    yr_data = {"yrend_in": yrend_in, "yrstart_mon": yrend_in + 10}

    y_list = Y.per.iloc[:m]
    est = ThresholdGenerator(grid=grid, length=length)
    thresholds, quantiles, x_quants, quantile_map, A = est.threshold_generation_given_seq(y_list, N, nIteration, gamma_values=gamma_vector)
    est = NonparametricBetaARChangePointTester(m, N, grid, thresholds, quantile_map, gamma_vector, A)
    results = est.run_test(Y.per)
    detected_point.append(results[0])

In [ ]:
nonparam_results = pd.DataFrame(detected_point)
nonparam_results.columns=["gamma0", "gamma025", "gamma04"]

In [ ]:
# Shewhart

In [ ]:
def generate_limits(y_list, C):
    mu_0 = np.mean(y_list)
    std_0 = np.std(y_list, ddof=1)
    tol = C*std_0
    lcl = mu_0-tol
    ucl = mu_0+tol
    
    return lcl, ucl

def detect_change_point(y_list, lcl, ucl):
    i = 0
    while i < len(y_list):
        if (y_list.iloc[i] >= lcl) and (y_list.iloc[i] <= ucl):
            i+=1
        else:
            return i
    print("No change point detected")

In [ ]:
m=400
shewhart_detected_point = []
for i in range(200, 300):
    if i%5==0:
        print(F"Processing the {str(i)} dataset.")
    Y = pd.read_csv(f"{os.getcwd()}/ds/Ha_X_datasets/X_seed_{str(i)}.csv")
    Y.columns = ["per"]
    limits = generate_limits(Y.iloc[:m].per, 3.24)
    # Detect change point
    location = detect_change_point(Y.iloc[m:].per, limits[0], limits[1])
    if location is not None:
        shewhart_detected_point.append(location)
    else:
        shewhart_detected_point.append(None)

In [ ]:
shewhart_results = pd.DataFrame(shewhart_detected_point)

In [ ]:
# EWMA

In [ ]:
def detect_change_point(y_list, lcl, ucl):
    for i in range(len(y_list)):
        if (y_list.iloc[i] >= lcl) and (y_list.iloc[i]<= ucl):
            pass
        else:
            return i
        
def normalize_time_series(data, target_variable_name, yr_data):
    y_list = data[data.year <= yr_data["yrend_in"]][target_variable_name]
    mu_0 = np.mean(y_list)
    std_0 = np.std(y_list)
    data[target_variable_name] = (data[target_variable_name] - mu_0) / std_0
    
    return data
    
def ewma_change_point_detection(data, target_variable_name, yr_data, shift, normalized=False):
    if normalized:
        data = normalize_time_series(data, target_variable_name, yr_data)
    smooth_factor, L = ewma_params[shift][0], ewma_params[shift][1]
    data['ewma'] = data[target_variable_name].ewm(alpha=smooth_factor, adjust=False).mean()

    adj_factor = np.sqrt((smooth_factor)/(2-smooth_factor))
    ucl = L*adj_factor
    lcl = -L*adj_factor
    signal_location = detect_change_point(data[data.year >= yr_data["yrstart_mon"]]["ewma"], lcl, ucl)
    return signal_location

In [ ]:
ewma_params = {"small": (0.05, 3.6), "medium": (0.14, 3.42), "large": (0.25, 3.35)}

In [ ]:
m=400
N= np.round(len(Y)/m-1, 2)
grid = 5
length = 10
nIteration = 500
gamma_vector = [0,0.25,0.4]
A = np.eye(grid)

ewma_detected_point = []
for i in range(200, 300):
    if i%5==0:
        print(F"Processing the {str(i)} dataset.")
    Y = pd.read_csv(f"{os.getcwd()}/ds/Ha_X_datasets/X_seed_{str(i)}.csv")
    Y.columns = ["per"]
    yrend_in = 2010
    Y["year"] = [yrend_in]*m + [yrend_in+10] * (len(Y)-m)

    yr_data = {"yrend_in": yrend_in, "yrstart_mon": yrend_in + 10}

    signal_location_small = ewma_change_point_detection(Y, "per", yr_data, shift="small", normalized=True)
    signal_location_medium = ewma_change_point_detection(Y, "per", yr_data, shift="medium", normalized=True)
    signal_location_large = ewma_change_point_detection(Y, "per", yr_data, shift="large", normalized=True)
    ewma_detected_point.append([signal_location_small, signal_location_medium, signal_location_large])

In [ ]:
ewma_results = pd.DataFrame(ewma_detected_point) + m
ewma_results.columns=["small", "medium", "large"]